In [5]:
import pandas as pd
import math
import os

os.chdir("../../../Mouse-Dynamics-Challenge/test_files/user12/")
listing = os.listdir('../../../Mouse-Dynamics-Challenge/test_files/user12/') 

#calculate distance between two points
def distance(x1,y1,x2,y2):
    return((pow(x2-x1,2)+pow(y2-y1,2))**(1.0/2))

#returns cps, avg. click time & no. of clicks
def clicks(df,data): 
    clicknum=0.01 #no. of clicks 
    avgclicktime=0; #average click time
    clickpersec=0; #clicks per sec

    for i in range(0, len(df)-1):
        if(df.iloc[i]['state']=='Pressed'):
            clicknum= clicknum+1
            if (df.iloc[i+1]['state']=='Released'):
                avgclicktime= avgclicktime+(df.iloc[i+1]['client timestamp'])-(df.iloc[i]['client timestamp'])
    avgclicktime=avgclicktime/clicknum
    clickpersec=clicknum/((df.iloc[len(df)-1]['client timestamp']-df.iloc[0]['client timestamp'])+0.01)
    data.append(clickpersec)
    data.append(avgclicktime)
    data.append(clicknum)

#calculate average speed of mouse
def speed(df,data):
    spsum=0 #average speed
    for i in range(0, len(df)-2):
        d=distance(df.iloc[i+1]['x'],df.iloc[i+1]['y'],df.iloc[i]['x'],df.iloc[i]['y'])
        t=(df.iloc[i+1]['client timestamp'])-(df.iloc[i]['client timestamp'])
        if (t!=0):
            spsum=spsum+(d/(t+0.01))
    spsum= spsum/((len(df)-2)+0.01)
    data.append(spsum)

#calculate avg. drag length, avg. drag speed, no. of drags & avg. drag time
def drag(df,data):
    draglen=0 #avg. drag length
    dragspeed=0 #avg. drag speed
    dragnum=0.01 #no. of drags
    dragtime=0 #avg. drag time
    for i in range(0,len(df)-2):
        if(df.iloc[i]['state']=='Pressed'and df.iloc[i+1]['state']=='Drag'):
            dis=0
            dragnum=dragnum+1
            it=df.iloc[i]['client timestamp']
            ix=df.iloc[i]['x']
            iy=df.iloc[i]['y']
            while(df.iloc[i]['state']!='Released'):
                fx=df.iloc[i+1]['x']
                fy=df.iloc[i+1]['y']
                dis=dis+distance(fx,fy,ix,iy)
                ix=fx
                iy=fy
                i= i+1
            ft=df.iloc[i]['client timestamp']
            draglen=draglen+dis
            dragtime=dragtime+(ft-it)
            dragspeed=dragspeed+(dis/((ft-it)+0.01))
    data.append(dragspeed/dragnum)
    data.append(dragnum)
    data.append(draglen/dragnum)
    data.append(dragtime/dragnum)
            
ans=[]
for i in range(0,10):
    infile=listing[i]
    df=pd.read_csv(infile)
    data=[]
    clicks(df,data)
    speed(df,data)
    drag(df,data)
    ans.insert(i,data)
dff = pd.DataFrame(ans,columns=['CPS','Avg click time','No.of clicks','Average mouse Speed','Avg drag speed','No. of drags','Avg drag length','Avg drag time'])
print(dff)

        CPS  Avg click time  No.of clicks  Average mouse Speed  \
0  0.175508        0.087605         63.01           368.584950   
1  0.229099        0.052689         21.01           227.819069   
2  0.092868        0.093052         59.01           333.267532   
3  0.185809        0.100901         81.01           276.109198   
4  0.249029        0.119234         59.01           247.040662   
5  0.153515        0.094545         79.01           245.471731   
6  0.065561        0.106948         40.01           292.068644   
7  0.081323        0.114459         65.01           388.095114   
8  0.059367        0.080552         42.01           250.779508   
9  0.287823        0.039638         37.01           138.639423   

   Avg drag speed  No. of drags  Avg drag length  Avg drag time  
0       94.782318          6.01       162.134175       0.757903  
1       99.553866         10.01       141.359915       1.292008  
2       34.779046          5.01        14.073888       0.323952  
3       6